# Task 2: Data Cleaning, Missing Observations and Feature Engineering

This project applies a fully structured preprocessing pipeline—a **quality-assurance framework** designed to stabilise messy Airbnb listing data, protect data integrity, and enable reliable, leakage-free modelling. Each stage enforces strict data standards, converts raw inputs into consistent formats, and produces a clean, reproducible feature set ready for machine-learning.

---

### **1. Protects Data Quality**

Handles inconsistent formats, missing values, skewed distributions, and unstructured text by:

* enforcing correct types for dates, rates, and numerics
* cleaning all text fields
* parsing lists (amenities, verifications)
* standardising geographic fields
* extracting structured review, bathroom, and host features

This prevents bad raw data from contaminating later steps.

---

### **2. Ensures Train/Test Consistency**

All engineered features follow the same rules across both datasets:

* Top-N amenities learned from **train only**
* One-hot encoding aligned to train columns
* TargetEncoder fitted only on train tiers
* Neighbourhood density mapped from train → test
* Outlier caps learned from train quantiles

Guarantees identical feature space and fair evaluation.

---

### **3. Prevents Data Leakage**

The pipeline avoids target contamination by:

* computing imputation stats on **training data only**
* learning hierarchical mappings from train only
* restricting price-tier encodings to train
* applying transformations without exposing test distribution

This preserves model validity and generalisation.

---

### **4. Modular and Reproducible**

Clear stage boundaries:

* Stage 1: Cleaning
* Stage 2: Imputation
* Stage 3: Encoding
* Stage 4: Feature Engineering
* Stage 5: Outlier Transformation

This modular structure makes the workflow easy to debug, transparent, and reproducible.



## Import Library

At first, I import all the libraries needed for the project.

In [1]:
# --- Standard Libraries ---
import re                           # Regular expressions for string cleaning
import time                         # Timing operations (e.g., geocoding loops)
import warnings                     # To suppress irrelevant warnings
warnings.filterwarnings('ignore')   # Silence warnings for cleaner output/logs

# --- Data Handling & Math ---
import pandas as pd                 # Core data manipulation library
import numpy as np                  # Numerical computing and vectorised operations

# --- Sentiment Analysis ---
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  
# Used to compute sentiment scores on text fields such as description or host_about

# --- Geocoding & Geospatial Processing ---
from geopy.geocoders import Nominatim     # For converting addresses → latitude/longitude
from geopy.point import Point             # Geopy coordinate object
import geopandas as gpd                   # Geospatial operations on GeoDataFrames
import osmnx as ox                        # OpenStreetMap (OSM) data download + routing
from shapely.geometry import Point         # Shapely geometric point objects
from sklearn.neighbors import BallTree     # Fast nearest-neighbour spatial search

# --- Feature Engineering / Encoding ---
from category_encoders import TargetEncoder  
# Encoding categorical values based on target mean (helps models handle high-cardinality categories)

# --- Utilities ---
from tqdm import tqdm                      # Progress bars for long loops (e.g., geocoding)


These libraries handle:
- data cleaning and manipulation (pandas, numpy)
- sentiment analysis for text features (VADER)
- geocoding and spatial processing (geopy, geopandas, osmnx, shapely, BallTree)
- feature encoding for high-cardinality categories (TargetEncoder)
- progress monitoring for long loops (tqdm)


In [ ]:
# from geopy.geocoders import Nominatim  # Import geocoder
# from geopy.point import Point  # Import geographic point object

# geolocator = Nominatim(user_agent='myGeocoder')  # Initialize geolocator with user agent

# def get_suburb_info(row):  # Define function to extract suburb or similar info
#     lat = row['latitude']
#     long = row['longitude']
#     location = geolocator.reverse(Point(lat, long), exactly_one=True, timeout=5)  # Reverse geocode
#     if location:
#         address = location.raw.get('address')  # Extract address dictionary
#         suburb = address.get('suburb')
#         town = address.get('town')
#         city_district = address.get('city_district')
#         village = address.get('village')
#         hamlet = address.get('hamlet')
#         if suburb: 
#             return suburb
#         elif town:
#             return town
#         elif city_district:
#             return city_district
#         elif village:
#             return village
#         elif hamlet:
#             return hamlet
#     return 0  # Return 0 if no valid result found

# df['suburb'] = df.apply(get_suburb_info, axis=1, result_type='expand')  # Apply function to all rows

# def get_council_info(row):  # Define function to extract municipality (council)
#     lat = row['latitude']
#     long = row['longitude']
#     location = geolocator.reverse(Point(lat, long), exactly_one=True, timeout=5)
#     if location:
#         address = location.raw.get('address')
#         municipality = address.get('municipality')  # Extract municipality field
#         return municipality
#     else:
#         return None  # Return None if not found

# df['municipality'] = df.apply(get_council_info, axis=1, result_type='expand')  # Apply function

# df_lookup = df[['latitude', 'longitude', 'suburb', 'municipality']].drop_duplicates()  # Keep only unique geocoded entries
# df_lookup.to_csv("suburb_municipality_lookup.csv", index=False)  # Save to CSV file


A reverse-geocoding module was implemented using Geopy to enrich each listing with accurate suburb and municipality information derived directly from latitude and longitude, replacing inconsistent Airbnb-provided location fields. A unique lookup table was generated to ensure reproducible, non-leaky, and efficient geographic enrichment across both the training and test datasets.

## Data Formatting

In [2]:
def clean_text_content(text):
    """
    Removes HTML, invisible characters, and collapses spaces.
    """
    if pd.isnull(text):
        return ''
    text = str(text)
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'\\n|\\r|\\t|&nbsp;|nbsp|br|\\xa0|&amp;', ' ', text)  # Invisible chars/entities
    text = re.sub(r'\s+', ' ', text)   # Collapse multiple spaces
    return text.strip()


def clean_amenities_string(x):
    """
    Parses amenity strings into clean lists.
    """
    x = str(x)
    x = re.sub(r'[\[\]{}"\']', '', x)  # Remove brackets and quotes
    return [a.strip().lower() for a in x.split(',') if a.strip()]

In this section, I define small helper functions used throughout the pipeline.

**`clean_text_content():`**
- Cleans raw text fields by removing HTML tags, invisible characters, and unnecessary whitespace.
- Ensures all text data is standardised before sentiment analysis or feature extraction.

**`clean_amenities_string():`**
- Converts the raw 'amenities' string into a clean list of individual items.
- Removes brackets, quotes, and extra spaces, and lowercases everything.
- Prepares the amenities field for feature engineering steps later on.


In [3]:
def _clean_price_and_rates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Owns: 'price', 'host_response_rate', 'host_acceptance_rate'
    """
    df = df.copy()

    # Price
    if 'price' in df.columns:
        df['price'] = df['price'].astype(str).str.replace(r'[\$,]', '', regex=True)
        df['price'] = pd.to_numeric(df['price'], errors='coerce')

    # Host rates
    for col in ['host_response_rate', 'host_acceptance_rate']:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace('%', '', regex=False)
                .replace('nan', np.nan)
            )
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col] / 100.0   # convert to proportion

    return df


def _process_bathrooms(df: pd.DataFrame) -> pd.DataFrame:
    """
    Owns: 'bathrooms', 'bath_type', 'bath_type_cleaned'
    """
    df = df.copy()

    if 'bathrooms' not in df.columns:
        return df

    # Bath type classification from original string
    df['bath_type'] = df['bathrooms'].astype(str).apply(
        lambda x: 'Shared' if 'shared' in x.lower() else
                  ('Private' if 'private' in x.lower() else 'Normal')
    )

    df['bath_type_cleaned'] = (
        df['bath_type']
        .fillna('unknown')
        .astype(str)
        .str.strip()
        .str.lower()
    )

    # Bathroom numeric extraction
    df['bathrooms'] = df['bathrooms'].astype(str).str.replace(
        'Half', '0.5', case=False, regex=False
    )
    df['bathrooms'] = df['bathrooms'].str.extract(r'(\d+\.?\d*)')[0]
    df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')

    return df


def _process_host_verifications(df: pd.DataFrame) -> pd.DataFrame:
    """
    Owns:
      'host_verifications', 'verifications_list',
      'email_verified', 'phone_verified', 'work_email_verified', 'num_verifications'
    """
    df = df.copy()

    if 'host_verifications' not in df.columns:
        return df

    df['host_verifications'] = df['host_verifications'].fillna('').astype(str)

    df['verifications_list'] = (
        df['host_verifications']
        .str.replace(r"[\[\]']", '', regex=True)
        .str.lower()
        .str.split(',')
    )

    df['verifications_list'] = df['verifications_list'].apply(
        lambda x: [i.strip() for i in x if i.strip()]
    )

    df['email_verified'] = df['verifications_list'].apply(lambda x: int('email' in x))
    df['phone_verified'] = df['verifications_list'].apply(lambda x: int('phone' in x))
    df['work_email_verified'] = df['verifications_list'].apply(lambda x: int('work_email' in x))
    df['num_verifications'] = df['verifications_list'].apply(len)

    return df


def _process_amenities(df: pd.DataFrame) -> pd.DataFrame:
    """
    Owns: 'amenities', 'amenities_list'
    """
    df = df.copy()

    if 'amenities' not in df.columns:
        return df

    df['amenities'] = df['amenities'].fillna('').astype(str)
    df['amenities_list'] = df['amenities'].apply(clean_amenities_string)

    # No counts/frequency here – that belongs to feature engineering stage.
    return df


def _process_dates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Owns: 'host_since', 'last_review'
    """
    df = df.copy()

    if 'host_since' in df.columns:
        df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')

    if 'last_review' in df.columns:
        df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    return df


def _process_text_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Owns: 'name', 'description', 'host_about', 'neighborhood_overview'
    """
    df = df.copy()

    text_cols = ['name', 'description', 'host_about', 'neighborhood_overview']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].apply(clean_text_content)

    return df


def _merge_geolocation(df: pd.DataFrame,
                       lookup_csv_path: str = "suburb_municipality_lookup.csv") -> pd.DataFrame:
    """
    Owns: 'suburb', 'municipality'
    (Assumes an 'ID' column exists in both df and the lookup CSV)
    """
    df = df.copy()
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    try:
        lookup_df = pd.read_csv(lookup_csv_path)
    except FileNotFoundError:
        print(f"Warning: '{lookup_csv_path}' not found. Skipping geolocation merge.")
        return df

    if 'ID' in df.columns and 'ID' in lookup_df.columns:
        df = df.merge(
            lookup_df[['ID', 'suburb', 'municipality']],
            on='ID',
            how='left'
        )

        if 'municipality' in df.columns:
            df['municipality'] = (
                df['municipality']
                .astype(str)
                .str.replace('City of ', '', regex=False)
                .str.replace('Shire of ', '', regex=False)
                .replace('nan', np.nan)
            )

    return df

**1. `_clean_price_and_rates(df)`**

- This function standardises the price and host percentage rate fields.

- Cleans the price column by removing $ and commas, then converts it to numeric.

- Cleans host_response_rate and host_acceptance_rate by removing %, handling "nan", converting to numbers, and dividing by 100 to convert percentages into proportions (e.g., 85% → 0.85).

- Ensures all rate-related columns are consistent and model-ready.

---

**2. `_process_bathrooms(df)`**

- This function extracts clean bathroom information and derives bathroom-type features.

- Creates a bath_type column by detecting keywords such as “shared” or “private”.

- Normalises bath type into bath_type_cleaned.

- Converts bathroom descriptions like "Half-bath" into numeric values (e.g., “Half” → 0.5).

- Extracts numeric quantities using regex and converts them to numeric type.

---

**3. `_process_host_verifications(df)`**

- This function converts the host verification string into structured features.

- Parses the raw host_verifications text into a clean list.

- Generates binary flags: email_verified, phone_verified, and work_email_verified.

- Creates num_verifications, counting how many verifications a host has.

- Helps models understand host trustworthiness using structured features.

--- 

**4. `_process_amenities(df)`**

- This function parses the raw amenities field.

- Converts the amenities string into a clean Python list using the earlier clean_amenities_string() helper.

---

**5. `_process_dates(df)`**

- This function converts important date fields into datetime objects.

- Parses host_since and last_review into proper datetime format.

- Prepares the columns for later calculations such as host experience or review recency.

---

**6. `_process_text_columns(df)`**

- This function cleans important text fields for later NLP/sentiment processing.

- Applies clean_text_content() to name, description, host_about, and neighborhood_overview.

- Removes HTML, invisible characters, and normalises spacing.

- Ensures clean text for sentiment analysis or keyword extraction.

---

**7. `_merge_geolocation(df)`**

- This function links the dataset with an external lookup table to enrich location fields.

- Converts latitude and longitude into numeric format.

- Merges a lookup CSV using ID (if available) to bring in suburb and municipality.

- Cleans municipality names by removing “City of” and “Shire of”.


In [4]:
def step_1_cleaning(df: pd.DataFrame,
                    lookup_csv_path: str = "suburb_municipality_lookup.csv") -> pd.DataFrame:
    """
    Stage 1 cleaning: calls non-overlapping helpers.

    Column ownership summary:
      - _clean_price_and_rates:      price, host_response_rate, host_acceptance_rate
      - _process_bathrooms:          bathrooms, bath_type, bath_type_cleaned
      - _process_host_verifications: host_verifications, verifications_list,
                                     email_verified, phone_verified,
                                     work_email_verified, num_verifications
      - _process_amenities:          amenities, amenities_list
      - _process_dates:              host_since, last_review
      - _process_text_columns:       name, description, host_about, neighborhood_overview
      - _merge_geolocation:          suburb, municipality
    """
    df_clean = df.copy()

    df_clean = _clean_price_and_rates(df_clean)
    df_clean = _process_bathrooms(df_clean)
    df_clean = _process_host_verifications(df_clean)
    df_clean = _process_amenities(df_clean)
    df_clean = _process_dates(df_clean)
    df_clean = _process_text_columns(df_clean)
    df_clean = _merge_geolocation(df_clean, lookup_csv_path=lookup_csv_path)

    return df_clean

**`step_1_cleaning`**

This function coordinates all individual cleaning helpers and ensures Stage 1 Cleaning is applied in the correct order. This step does not include its own cleaning logic; instead, it sequentially calls each specialised helper function. Every helper is responsible for specific columns, ensuring there is no overlap or accidental reprocessing.<br>

Functions called in this stage:

- `_clean_price_and_rates()` → cleans price and host rate percentages

- `_process_bathrooms()` → extracts bathroom counts and bathroom type

- `_process_host_verifications()` → parses verification lists and creates binary flags

- `_process_amenities()` → converts amenities into clean lists

- `_process_dates()` → converts date fields to datetime

- `_process_text_columns()` → cleans text fields for NLP

- `_merge_geolocation()` → merges in suburb/municipality lookup information

---

This function separates responsibilities to prevent column conflicts, applies consistent cleaning across both train and test datasets, and maintains a modular, easy-to-debug preprocessing pipeline.


## Data Imputation

These functions improve data completeness by inferring missing types, location, and availability from existing signals in the dataset.

In [5]:
def infer_missing_types_logic(df):
    """
    Infers property_type and room_type from the 'description' text 
    using keyword matching (Regex).
    """
    df = df.copy()
    
    # --- A. Infer Property Type ---
    if 'description' in df.columns and 'property_type' in df.columns:
        desc = df['description'].fillna('').str.lower()
        conditions = [
            desc.str.contains('villa'),
            desc.str.contains('cabin|cottage|barn|hut|yurt|chalet'),
            desc.str.contains('apartment|flat|unit|condo'),
            desc.str.contains('studio|loft|tiny home'),
            desc.str.contains('farm|nature lodge'),
            desc.str.contains('tent|camper|rv|boat|train|tower'),
            desc.str.contains('entire home|entire house'),
            desc.str.contains('private room'),
            desc.str.contains('shared room|bunk'),
        ]
        choices = [
            'Entire villa', 'Entire cabin', 'Entire rental unit', 'Entire studio',
            'Farm stay', 'Camper/RV', 'Entire home', 'Private room in home',
            'Shared room in rental unit'
        ]
        # Infer and Fill
        inferred_prop = np.select(conditions, choices, default=None)
        # Only fill missing values
        df['property_type'] = df['property_type'].fillna(pd.Series(inferred_prop))

    # --- B. Infer Room Type ---
    if 'description' in df.columns and 'room_type' in df.columns:
        desc = df['description'].fillna('').str.lower()
        conditions_room = [
            desc.str.contains(r'\b(entire|whole)\b.*\b(home|apartment|house|flat|unit)\b', regex=True),
            desc.str.contains(r'\b(private room|private bedroom|own room)\b', regex=True),
            desc.str.contains(r'\b(shared room|bunk bed|dormitory|shared)\b', regex=True),
            desc.str.contains(r'\b(hotel|suite|serviced apartment|aparthotel)\b', regex=True),
        ]
        choices_room = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
        
        # Infer and Fill
        inferred_room = np.select(conditions_room, choices_room, default='Unknown')
        inferred_series = pd.Series(inferred_room).replace('Unknown', np.nan)
        df['room_type'] = df['room_type'].fillna(inferred_series)

    return df

**`infer_missing_types_logic(df)`**

This function infers missing `property_type` and `room_type` values from the `description` text using keyword-based rules (regex-style matching).

* For **`property_type`**, it scans the description for words like *villa*, *cabin*, *apartment*, *studio*, *farm*, *camper/RV*, *private room*, or *shared room*, and maps them to consistent property type labels.
* It only fills values where `property_type` is missing, so existing values are not overwritten.
* For **`room_type`**, it looks for phrases like *entire home/apartment*, *private room*, *shared room*, or *hotel/suite*, and infers a standardised room type.
* Inferred values are used only to fill missing `room_type`, keeping any original entries intact.

Overall, this function enriches incomplete listings by using textual clues in the description to recover likely property and room types.

In [6]:
def impute_neighbourhood_from_geo(df):
    """
    Reproduce old behaviour:
    - neighbourhood = suburb
    - neighbourhood_cleansed filled from municipality when missing
    """
    df = df.copy()

    if 'suburb' in df.columns:
        # df['neighbourhood'] = df['suburb']
        # To match exactly, do the same:
        df['neighbourhood'] = df['suburb']

    if 'municipality' in df.columns and 'neighbourhood_cleansed' in df.columns:
        df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].fillna(df['municipality'])

    return df


**`impute_neighbourhood_from_geo(df)`**

This function reconstructs the old behaviour for neighbourhood-related fields using geolocation features.

* Sets `neighbourhood` equal to `suburb`, matching the original pipeline logic.
* Fills missing `neighbourhood_cleansed` values using `municipality` where available.

This keeps the location-related variables consistent with the previous implementation while leveraging suburb and municipality information to reduce missingness.


In [7]:
def impute_availability_logic(df):
    """
    Fills 'availability_365' using 'availability_90' * 4 if 365 is missing.
    """
    df = df.copy()
    if 'availability_365' in df.columns and 'availability_90' in df.columns:
        mask = df['availability_365'].isna() & df['availability_90'].notna()
        df.loc[mask, 'availability_365'] = df.loc[mask, 'availability_90'] * 4
    return df

**`impute_availability_logic(df)`**

This function imputes missing `availability_365` values using `availability_90`.

* Where `availability_365` is missing but `availability_90` is present, it sets:
  **`availability_365 = availability_90 * 4`**
* This assumes that the 90-day availability pattern roughly scales across the full year.

In [8]:
def stage_2_imputation(train_df, test_df):
    """
    Main Imputation Orchestrator.
    1. Fits stats (Mode/Median/Skew) on TRAIN only.
    2. Transforms both TRAIN and TEST using those stats.
    """
    print("--- Stage 2: Processing missing values ---")
    
    # Create copies to avoid SettingWithCopy warnings
    train_df = train_df.copy()
    test_df = test_df.copy()

    # ---------------------------------------------------------
    # STEP A: LOGIC-BASED INFERENCE (Row-by-Row)
    # ---------------------------------------------------------
    # Apply to each dataset and write back
    train_df = infer_missing_types_logic(train_df)
    train_df = impute_availability_logic(train_df)
    train_df = impute_neighbourhood_from_geo(train_df)

    test_df = infer_missing_types_logic(test_df)
    test_df = impute_availability_logic(test_df)
    test_df = impute_neighbourhood_from_geo(test_df)

    # ---------------------------------------------------------
    # STEP B: TEXT PLACEHOLDERS
    # ---------------------------------------------------------
    text_placeholders = {
        'name': 'Unnamed Listing',
        'description': 'No description provided',
        'neighborhood_overview': 'No neighborhood overview',
        'host_about': 'No host description'
    }
    for col, fallback in text_placeholders.items():
        if col in train_df.columns: train_df[col] = train_df[col].fillna(fallback)
        if col in test_df.columns: test_df[col] = test_df[col].fillna(fallback)

    # ---------------------------------------------------------
    # STEP C: FIT (LEARN) STATISTICS FROM TRAINING DATA
    # ---------------------------------------------------------
    stats_map = {}
    
    # 1. Categorical Modes (Learn from Train)
    cat_cols = ['host_location', 'host_neighbourhood', 'host_response_time', 'host_is_superhost']
    for col in cat_cols:
        if col in train_df.columns:
            stats_map[col] = train_df[col].mode().iloc[0]

    # 2. Numerical Skew-Aware Stats (Learn from Train)
    num_cols = [
        'host_response_rate', 'host_acceptance_rate', 'bathrooms', 'bedrooms', 'beds',
        'minimum_minimum_nights', 'maximum_maximum_nights', 'review_scores_rating', 
        'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
        'review_scores_communication', 'review_scores_location', 'review_scores_value', 
        'reviews_per_month'
    ]
    for col in num_cols:
        if col in train_df.columns:
            skewness = train_df[col].skew()
            # If symmetric (skew < 0.5), use Mean. Else use Median.
            if abs(skewness) < 0.5:
                stats_map[col] = train_df[col].mean()
            else:
                stats_map[col] = train_df[col].median()

    # 3. Hierarchical Map (Property Type -> Room Type Mode)
    # Learn which room_type is most common for each property_type in TRAIN
    if 'property_type' in train_df.columns and 'room_type' in train_df.columns:
        room_mode_map = (
            train_df.groupby('property_type')['room_type']
            .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
            .to_dict()
        )
        # Also get global defaults from TRAIN
        global_prop_mode = train_df['property_type'].mode()[0]
        global_room_mode = train_df['room_type'].mode()[0]
    else:
        room_mode_map = {}
        global_prop_mode, global_room_mode = "Entire rental unit", "Entire home/apt"

    # ---------------------------------------------------------
    # STEP D: TRANSFORM (APPLY) TO BOTH DATASETS
    # ---------------------------------------------------------
    
    def apply_imputation(df):
        # 1. Apply Simple Stats (Cat & Num) using the stats_map learned from Train
        for col, value in stats_map.items():
            if col in df.columns:
                df[col] = df[col].fillna(value)
        
        # 2. Apply Hierarchical (Prop -> Room)
        if 'room_type' in df.columns:
            def fill_room(row):
                if pd.isna(row['room_type']):
                    return room_mode_map.get(row['property_type'], np.nan)
                return row['room_type']
            df['room_type'] = df.apply(fill_room, axis=1)
        
        # 3. Final Fallback for Prop/Room (Global Modes from Train)
        if 'property_type' in df.columns:
            df['property_type'] = df['property_type'].fillna(global_prop_mode)
        if 'room_type' in df.columns:
            df['room_type'] = df['room_type'].fillna(global_room_mode)

        # 4. Date Imputation (Sort -> FFill/BFill)
        # Impute dates internally within the dataset (no cross-dataset leaking)
        if 'ID' in df.columns:
            df = df.sort_values(by='ID')
            if 'first_review' in df.columns:
                df['first_review'] = df['first_review'].fillna(method='ffill')
            if 'last_review' in df.columns:
                df['last_review'] = df['last_review'].fillna(method='bfill')
            df = df.reset_index(drop=True)
            
        return df

    # Apply the logic
    train_filled = apply_imputation(train_df)
    test_filled = apply_imputation(test_df)
    
    print("--- Stage 2: Imputation Complete ---")
    return train_filled, test_filled

**`stage_2_imputation`**

This function manages all missing-value handling for both the training and test datasets.
It learns imputation rules **only from the training set**, then applies them consistently to both datasets to avoid data leakage.

**1. Logic-based inference (row-level fixes)**

Applies rule-based functions that infer missing values directly from existing information:

* `infer_missing_types_logic()` → infers `property_type` and `room_type` from the description text
* `impute_availability_logic()` → imputes yearly availability from 90-day availability
* `impute_neighbourhood_from_geo()` → restores neighbourhood logic using suburb and municipality

**2. Text fallbacks**

Fills missing text fields with clear placeholders such as *“Unnamed Listing”* or *“No description provided”*.

**3. Learn statistics from the training dataset**

Builds an imputation dictionary using training-only data:

* Categorical modes (most frequent values)
* Numerical replacements based on skew-aware logic

  * If the distribution is symmetric → use mean
  * If skewed → use median
* Hierarchical mapping:
  **property_type → most common room_type**

**4. Apply learned imputations to both train and test**

Uses the training statistics to fill missing values consistently:

* Applies the learned categorical and numerical values
* Fills room types using the property-type hierarchy
* Falls back to global modes where needed

**5. Internal date imputation**

Sorts rows by `ID` and uses forward/backward fill to impute missing dates like `first_review` and `last_review` within each dataset independently.

---
This design prevents data leakage by learning imputations only from the training data, ensures consistent behaviour across both datasets, blends rule-based inference with statistical methods, and maintains a modular structure that aligns cleanly with the overall multi-stage pipeline.





## Data Encoding

In [9]:
def encode_amenities(train_df, test_df, top_n=10):
    """
    Learns top N amenities from TRAIN, creates binary columns in both.
    """
    # 1. Analyze Train to get Top N
    train_exploded = train_df.explode('amenities_list')
    top_amenities = train_exploded['amenities_list'].value_counts().head(top_n).index.tolist()
    
    # 2. Apply to both
    for df in [train_df, test_df]:
        for amenity in top_amenities:
            col_name = f'has_{amenity.replace(" ", "_").replace("-", "_")}'
            # Check list column exists before applying
            if 'amenities_list' in df.columns:
                df[col_name] = df['amenities_list'].apply(lambda x: int(amenity in x))
    
    return train_df, test_df

**`encode_amenities(train_df, test_df)`**

This function identifies the top N most common amenities from the training dataset and creates matching binary indicator columns in both datasets.

1. It first explodes the amenities_list column in the training set and selects the top N most frequent amenities.

2. For each of these amenities, it adds a new column such as has_wifi or has_heating, where values are 1 if the amenity is present and 0 otherwise.

3. The same columns are then created in the test set to ensure the feature space is aligned across both datasets.

Overall, this function converts unstructured amenity lists into structured that capture the popularity and presence of key amenities.

In [10]:
def optimized_property_type_encoding(df_train, df_test, price_col='price'):
    """
    Advanced Property Type Encoding: Tiers, Grouping, and Target Encoding.
    """
    # Avoid SettingWithCopy
    df_train = df_train.copy()
    df_test = df_test.copy()
    
    # --- A. Price Tiers (for reference) ---
    df_train['price_tier'] = pd.cut(
        df_train[price_col], bins=[0, 100, 500, float('inf')],
        labels=['Low', 'Mid', 'High'], right=False
    )

    # --- B. Tier Scores ---
    # Learn high/mid lists from Train
    high_tier_props = df_train[df_train['price_tier'] == 'High']['property_type'].value_counts().index.tolist()
    mid_tier_props = df_train[df_train['price_tier'] == 'Mid']['property_type'].value_counts().index.tolist()

    def compute_tier_score(prop_type):
        if prop_type in high_tier_props[:10]: return 3
        elif prop_type in mid_tier_props[:15]: return 2
        elif prop_type in high_tier_props[10:20]: return 1.5
        else: return 1

    for df in [df_train, df_test]:
        df['property_tier_score'] = df['property_type'].apply(compute_tier_score)

    # --- C. Groupings ---
    property_groups = {
        'luxury_homes': ['Entire villa', 'Entire serviced apartment', 'Farm stay'],
        'unique_stays': ['Entire cabin', 'Earthen home', 'Barn', 'Castle'],
        'standard_rentals': ['Entire rental unit', 'Entire home', 'Private room in home'],
        'budget_options': ['Shared room in rental unit', 'Shared room in hostel']
    }
    for group, props in property_groups.items():
        for df in [df_train, df_test]:
            df[f'is_{group}'] = df['property_type'].isin(props).astype(int)
            df[f'{group}_tier'] = df[f'is_{group}'] * df['property_tier_score']

    # --- D. Target Encoding ---
    # Map tier to numeric for target encoding
    target_mapping = {'High': 3, 'Mid': 2, 'Low': 1}
    y_target = df_train['price_tier'].map(target_mapping).fillna(1) # Fill NaN with Low
    
    encoder = TargetEncoder(cols=['property_type'], smoothing=20)
    df_train['property_type_encoded'] = encoder.fit_transform(df_train['property_type'], y_target)
    df_test['property_type_encoded'] = encoder.transform(df_test['property_type'])

    # --- E. Rare Premium Flag ---
    rare_premium = df_train.groupby('property_type').filter(
        lambda x: (x[price_col].mean() > 300) & (len(x) < 50)
    )['property_type'].unique()

    for df in [df_train, df_test]:
        for pt in rare_premium:
            col = f'is_{pt.replace(" ", "_").replace("/", "_").lower()}'
            df[col] = (df['property_type'] == pt).astype(int)
            
    # Cleanup temporary column
    df_train = df_train.drop(columns=['price_tier'])
    
    return df_train, df_test

**`optimized_property_type_encoding(df_train, df_test)`**

This function builds a rich set of features based on property_type, combining tiering, grouping, rare-category detection, and target encoding.

1. It starts by categorizing each listing into price tiers (Low, Mid, High), learned from the training set.

2. Based on these tiers, it learns which property types are most common in high-priced and mid-priced listings, creating a property_tier_score to reflect premium-level property types.

3. It groups property types into meaningful categories (e.g., luxury homes, unique stays, standard rentals, budget options) and generates binary flags and weighted tier scores for each group.

4. Target encoding is applied to property_type using price tiers, allowing the model to capture complex relationships between property type and pricing.

5. It also detects “rare premium” property types (high average price but low frequency) and adds explicit flags for those categories.

Overall, this function transforms raw property types into a comprehensive set of numerical and categorical indicators that help the model understand pricing signals associated with different property categories.

In [11]:
def encode_neighbourhood_features(df_train, df_test):
    """
    Advanced Neighbourhood Encoding: Popularity Ratios, Premium Flags, Target Encoding.
    """
    df_train = df_train.copy()
    df_test = df_test.copy()

    # --- A. Setup Price Tiers (Train Only) ---
    df_train['price_tier'] = pd.cut(
        df_train['price'], bins=[0, 100, 500, float('inf')],
        labels=['Low', 'Mid', 'High'], right=False
    )

    # --- B. Tier Popularity Scores ---
    tier_popularity = df_train.groupby(['neighbourhood_cleansed', 'price_tier']).size().unstack()
    tier_popularity = tier_popularity.div(tier_popularity.sum(axis=1), axis=0) # Normalize

    for tier in ['High', 'Mid', 'Low']:
        mapper = tier_popularity[tier].fillna(0)
        df_train[f'neighbourhood_{tier.lower()}_score'] = df_train['neighbourhood_cleansed'].map(mapper)
        df_test[f'neighbourhood_{tier.lower()}_score'] = df_test['neighbourhood_cleansed'].map(mapper).fillna(0)

    # --- C. Premium & Rare Flags ---
    high_ratio = tier_popularity['High'] / tier_popularity.sum(axis=1)
    premium_neighs = high_ratio[high_ratio > 0.3].index.tolist()
    
    rare_neighs = df_train['neighbourhood_cleansed'].value_counts()
    rare_list = rare_neighs[rare_neighs < 50].index

    for df in [df_train, df_test]:
        df['is_premium_neighbourhood'] = df['neighbourhood_cleansed'].isin(premium_neighs).astype(int)
        df['is_rare_neighbourhood'] = df['neighbourhood_cleansed'].isin(rare_list).astype(int)

    # --- D. Target Encoding ---
    target_mapping = {'High': 3, 'Mid': 2, 'Low': 1}
    y_target = df_train['price_tier'].map(target_mapping).fillna(1)

    encoder = TargetEncoder(cols=['neighbourhood_cleansed'], smoothing=15)
    df_train['neighbourhood_encoded'] = encoder.fit_transform(df_train['neighbourhood_cleansed'], y_target)
    df_test['neighbourhood_encoded'] = encoder.transform(df_test['neighbourhood_cleansed'])

    # Cleanup
    df_train = df_train.drop(columns=['price_tier'])

    return df_train, df_test

**`encode_neighbourhood_features(df_train, df_test)`**

This function creates advanced neighbourhood-level features by analyzing price tiers, popularity ratios, rarity, and encoded signals.

1. It begins by assigning price tiers (Low, Mid, High) to each training listing and computes how frequently each neighbourhood appears in each price tier.

2. These ratios are then used to create neighbourhood-level popularity scores, such as neighbourhood_high_score or neighbourhood_mid_score, for both train and test sets.

3. The function identifies premium neighbourhoods (areas with a high proportion of expensive listings) and rare neighbourhoods (areas with very few listings), adding binary flags for both.

4. Target encoding is applied to neighbourhood_cleansed, enabling the model to capture nuanced pricing effects tied to different local areas.

Overall, this function enriches the dataset with detailed neighbourhood indicators, helping the model better recognise location-based pricing patterns.

In [12]:
def stage_3_encoding(train_df, test_df):
    """
    Main pipeline to execute all encoding steps.
    """
    print("--- Stage 3: Encoding Started ---")
    
    # 1. Binary Columns (Map t/f to 1/0)
    binary_cols = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 
                   'host_identity_verified', 'has_availability']
    for df in [train_df, test_df]:
        for col in binary_cols:
            if col in df.columns:
                df[col] = df[col].map({'t': 1, 'f': 0}).fillna(0).astype(int)

    # 2. Ordinal Encoding (Response Time)
    response_mapping = {
        'a few days or more': 0, 'within a day': 1, 
        'within a few hours': 2, 'within an hour': 3
    }
    for df in [train_df, test_df]:
        if 'host_response_time' in df.columns:
            df['host_response_time_encoded'] = (
                df['host_response_time'].map(response_mapping).fillna(-1).astype(int)
            )

    # 3. Amenities Encoding (Top N)
    train_df, test_df = encode_amenities(train_df, test_df, top_n=10)

    # 4. Host Location & Host Neighbourhood (Simple Top-K One Hot)
    # We clean, keep top 5, and then One-Hot encode
    def clean_loc(x):
        return 'other' if (pd.isna(x) or ',' in str(x)) else x

    for col, prefix in [('host_neighbourhood', 'neigh'), ('host_location', 'loc')]:
        if col in train_df.columns:
            # Clean
            train_df[f'{col}_cleaned'] = train_df[col].apply(clean_loc)
            test_df[f'{col}_cleaned'] = test_df[col].apply(clean_loc)
            
            # Identify Top 5 from TRAIN
            top_5 = train_df[f'{col}_cleaned'].value_counts().nlargest(5).index.tolist()
            
            # Encode (Keep Top 5, else 'other')
            for df in [train_df, test_df]:
                df[f'{col}_encoded_label'] = df[f'{col}_cleaned'].apply(lambda x: x if x in top_5 else 'other')
            
            # One-Hot Encode (Get Dummies)
            # Note: We do this separately to ensure alignment, then concat
            train_dummies = pd.get_dummies(train_df[f'{col}_encoded_label'], prefix=prefix, drop_first=True)
            test_dummies = pd.get_dummies(test_df[f'{col}_encoded_label'], prefix=prefix, drop_first=True)
            
            # Align columns (Ensure Test has same columns as Train)
            test_dummies = test_dummies.reindex(columns=train_dummies.columns, fill_value=0)
            
            # Concat back
            train_df = pd.concat([train_df, train_dummies.astype(int)], axis=1)
            test_df = pd.concat([test_df, test_dummies.astype(int)], axis=1)

    # 5. Bathroom Type (One Hot)
    if 'bath_type_cleaned' in train_df.columns:
        train_dummies = pd.get_dummies(train_df['bath_type_cleaned'], prefix='bath_type')
        test_dummies = pd.get_dummies(test_df['bath_type_cleaned'], prefix='bath_type')
        
        # Align columns
        test_dummies = test_dummies.reindex(columns=train_dummies.columns, fill_value=0)
        
        train_df = pd.concat([train_df, train_dummies.astype(int)], axis=1)
        test_df = pd.concat([test_df, test_dummies.astype(int)], axis=1)

    # 6. Advanced Property & Neighbourhood Encoding
    train_df, test_df = optimized_property_type_encoding(train_df, test_df)
    train_df, test_df = encode_neighbourhood_features(train_df, test_df)

    print("--- Stage 3: Encoding Complete ---")
    return train_df, test_df

**`stage_3_encoding(train_df, test_df)`** 

This function applies all feature-encoding steps to transform cleaned and imputed data into model-ready numeric features for both the training and test datasets.

1. First, it converts several binary text columns (`instant_bookable`, `host_is_superhost`, `host_has_profile_pic`, `host_identity_verified`, `has_availability`) from `'t'/'f'` strings into `1/0` integers, ensuring these yes/no fields are usable in numerical models.

2. Next, it performs ordinal encoding on `host_response_time` by mapping categories such as *“a few days or more”*, *“within a day”*, *“within a few hours”*, and *“within an hour”* to ordered integer scores, so the model can understand the speed of host response as a ranked variable.

3. It then calls `encode_amenities()` to learn the top N most common amenities from the training data and create corresponding binary features (e.g., `has_wifi`) in both train and test, turning the unstructured `amenities_list` into structured indicator variables.

4. For `host_neighbourhood` and `host_location`, it cleans the values, identifies the top 5 most frequent categories from the training set, and one-hot encodes those while grouping all other values into an `"other"` category. It also aligns dummy columns between train and test so they share the same encoded structure.

5. It applies one-hot encoding to `bath_type_cleaned`, creating separate columns for each bathroom type and ensuring that train and test have aligned bathroom-type dummy variables.

* Finally, it calls `optimized_property_type_encoding()` and `encode_neighbourhood_features()` to generate advanced property-type and neighbourhood-based features, including tier scores, group flags, popularity ratios, premium/rare flags, and target-encoded variables.
---
Overall, this function centralises all encoding logic, ensuring that both the training and test datasets receive consistent, well-structured numeric features that are ready to be used by machine learning models.


## Data Engineering

In [13]:
def haversine_distance(lat1, lon1, lat2=-37.8136, lon2=144.9631):
    """Calculates distance (km) between lat/lon and Melbourne CBD."""
    R = 6371  # Earth radius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    d_phi = np.radians(lat2 - lat1)
    d_lambda = np.radians(lon2 - lon1)
    a = np.sin(d_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(d_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

**`haversine_distance(lat1, lon1, lat2=-37.8136, lon2=144.9631)`** 

This function computes the geodesic (Haversine) distance in kilometres between a listing’s coordinates and the Melbourne CBD.

1. It converts latitude and longitude values from degrees to radians.

2. Applies the Haversine formula to account for Earth’s curvature.

3. Returns the straight-line distance in kilometres, using Melbourne CBD (−37.8136, 144.9631) as the default reference point.

Overall, this function generates a useful geographic feature that captures how far each listing is from a central, high-demand location.

In [14]:
def get_vader_score(text, analyzer):
    """Returns compound sentiment score (0 if missing)."""
    if pd.isnull(text) or str(text).strip() == '':
        return 0
    return analyzer.polarity_scores(str(text))['compound']

**`get_vader_score(text, analyzer)`** 

This function produces a VADER compound sentiment score for text fields such as listing descriptions or host bios.

1. If the text is missing or empty, it returns 0 as a neutral baseline.

2. Otherwise, it uses the VADER sentiment analyzer to compute the compound score, which ranges from −1 (very negative) to +1 (very positive).

This allows the model to capture emotional tone or marketing quality in descriptions as a numerical feature.

In [15]:
def count_amenities_list(amenities_str, reference_list):
    """Counts how many reference items appear in the amenities string."""
    if pd.isna(amenities_str): return 0
    clean_str = str(amenities_str).lower().replace('{', '').replace('}', '').replace('"', '')
    return sum(1 for item in reference_list if item.lower() in clean_str)


**`count_amenities_list(amenities_str, reference_list)`** 

This function counts how many items from a predefined reference list (e.g., “top 25 amenities”) appear in the raw amenities string.

1. Handles missing values by returning 0.

2. Cleans the raw amenities text by removing brackets and quotes.

3. Checks each reference amenity against the cleaned string and counts how many are present.

This helps quantify amenity richness in a simple numeric way, especially when extracting important amenities from messy text.

In [16]:
def add_basic_structural_features(df):
    df = df.copy()
    
    # --- Review & Host Dates ---
    review_cols = [
        'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
        'review_scores_communication', 'review_scores_location', 'review_scores_value'
    ]
    # Calculate avg/min if columns exist
    present_cols = [c for c in review_cols if c in df.columns]
    if present_cols:
        df['review_scores_avg'] = df[present_cols].mean(axis=1)
        df['review_scores_min'] = df[present_cols].min(axis=1)

    if 'number_of_reviews' in df.columns:
        df['review_count_log'] = np.log1p(df['number_of_reviews'])

    # Host Years & Days Since Last Review

    # Ensure columns exist so later steps don't break
    if 'host_since' not in df.columns:
        df['host_since'] = pd.NaT
    if 'last_review' not in df.columns:
        df['last_review'] = pd.NaT

    # Step 1: Convert date columns to datetime format
    df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    # Step 2: Use the latest date in the dataset as a reference point
    reference_date = max(df['host_since'].max(), df['last_review'].max())

    # Step 3: Calculate host experience in years
    df['host_years'] = (reference_date - df['host_since']).dt.days / 365

    # Step 4: Fill missing host_since values with 0 (no experience)
    df['host_years'] = df['host_years'].fillna(0)

    # --- Step 1: Convert 'last_review' to datetime ---
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    # --- Step 2: Define reference date as the latest valid review date ---
    reference_date = df['last_review'].max()

    # --- Step 3: Calculate time since last review ---
    df['days_since_last_review'] = (reference_date - df['last_review']).dt.days

    # --- Step 4: Fill missing with max to represent "never reviewed" or very old ---
    df['days_since_last_review'] = df['days_since_last_review'].fillna(
        df['days_since_last_review'].max()
    )

    # Reapplied to reinforce datetime consistency and ensure no downstream overwrite
    # corrupted the time-based feature; this keeps review-based features stable and reproducible.

    # --- Step 1: Convert 'last_review' to datetime ---
    df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

    # --- Step 2: Define reference date as the latest valid review date ---
    reference_date = df['last_review'].max()

    # --- Step 3: Calculate time since last review ---
    df['days_since_last_review'] = (reference_date - df['last_review']).dt.days

    # --- Step 4: Fill missing with max to represent "never reviewed" or very old ---
    df['days_since_last_review'] = df['days_since_last_review'].fillna(
        df['days_since_last_review'].max()
    )

    # --- Property Structure ---
    df['is_entire_home'] = (df['room_type'] == 'Entire home/apt').astype(int)
    df['is_hotel_room'] = df['room_type'].astype(str).str.lower().str.contains('hotel').astype(int)
    
    if 'bedrooms' in df.columns:
        df['entire_home_x_bedrooms'] = df['is_entire_home'] * df['bedrooms']

    if 'beds' in df.columns and 'accommodates' in df.columns:
        df['beds_per_guest'] = np.log1p(df['beds'] / df['accommodates'].replace(0, 1))

    return df

**`add_basic_structural_features(df)`**

This function engineers core structural features about reviews, host experience, and listing configuration.

1. It first looks for individual review score columns (accuracy, cleanliness, check-in, communication, location, value). If they exist, it summarises them into two aggregate metrics:

  * `review_scores_avg` – the average of available review scores per listing
  * `review_scores_min` – the minimum review score per listing, capturing the weakest area of guest feedback.

2. If `number_of_reviews` is present, it creates `review_count_log` using a log(1 + x) transformation to stabilise the skew often seen in review counts while preserving the relative ordering between listings.

3. For host experience and review recency, it first ensures `host_since` and `last_review` exist, converts them to datetime, and uses the latest available date in the dataset as a reference point to compute `host_years`, which measures how long the host has been active in years, filling missing values with 0 to represent no experience.

4. It then computes `days_since_last_review` as the number of days since the most recent valid review date, filling missing values with the maximum observed value to represent listings that have never been reviewed or have very old reviews. This calculation is intentionally reapplied to reinforce datetime consistency and protect this time-based feature from any accidental overwrites in later steps.

5. Finally, it adds structure-related features:

  * `is_entire_home` – a flag for listings where `room_type` is “Entire home/apt”
  * `is_hotel_room` – a flag for listings whose room type text contains “hotel”
  * `entire_home_x_bedrooms` – an interaction capturing how many bedrooms entire homes have
  * `beds_per_guest` – a log-transformed beds-per-accommodated-guest ratio, measuring how generous the sleeping setup is relative to capacity.

Overall, this function compresses raw review, host, and structural information into clean, numeric features that are highly informative for downstream price modelling.


In [17]:
def add_text_features(df):
    df = df.copy()
    
    # --- 1. Word/Char Counts  ---
    if 'description' in df.columns:
        df['desc_word_count'] = df['description'].astype(str).apply(lambda x: len(x.split()))
        df['desc_char_count'] = df['description'].astype(str).apply(len)
    
    if 'host_about' in df.columns:
        df['host_about_word_count'] = df['host_about'].astype(str).apply(lambda x: len(x.split()))
        
    if 'neighborhood_overview' in df.columns:
        df['neigh_word_count'] = df['neighborhood_overview'].astype(str).apply(lambda x: len(x.split()))

    # --- 2. Sentiment (VADER) ---
    analyzer = SentimentIntensityAnalyzer()
    df['desc_sentiment'] = df['description'].astype(str).apply(lambda x: get_vader_score(x, analyzer))
    df['host_sentiment'] = df['host_about'].astype(str).apply(lambda x: get_vader_score(x, analyzer))
    df['neigh_sentiment'] = df['neighborhood_overview'].astype(str).apply(lambda x: get_vader_score(x, analyzer))

    # --- 3. Luxury Keywords (Full List) ---
    luxury_keywords = [
        'luxury', 'executive', 'beachfront', 'renovated', 'designer', 'stunning', 'premium',
        'elegant', 'spacious', 'exclusive', 'high-end', 'retreat', 'resort', 'deluxe', 
        'panoramic', 'modern', 'gourmet', 'spa', 'oceanview', 'sunset view', 
        'chic', 'newly built', 'private pool', 'highrise', 'skyline view', 'penthouse', 
        'architect-designed', 'boutique', 'five-star', 'platinum', 'expansive'
    ]
    
    # Regex cleaning exactly as requested
    df['combined_text'] = (df['name'].astype(str) + ' ' + df['description'].astype(str)).str.lower()
    df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'[^a-z0-9\s\-]', ' ', x))
    df['combined_text'] = df['combined_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

    keyword_hits = []
    for kw in luxury_keywords:
        pattern = rf'\b{re.escape(kw)}\b'
        col = f'has_kw_{kw.replace(" ", "_")}'
        df[col] = df['combined_text'].str.contains(pattern, regex=True).astype(int)
        keyword_hits.append(df[col])

    df['luxury_keyword_hits'] = sum(keyword_hits)
    df = df.drop(columns=['combined_text']) # Cleanup
    
    return df

**`add_text_features(df)`**

This function extracts length, sentiment, and luxury-related signals from listing text fields.

1. For `description`, `host_about`, and `neighborhood_overview`, it creates word-count and character-count style metrics such as `desc_word_count`, `desc_char_count`, `host_about_word_count`, and `neigh_word_count`, which capture how detailed each text field is.

2. It then uses VADER to compute sentiment scores for descriptions, host bios, and neighbourhood overviews (`desc_sentiment`, `host_sentiment`, `neigh_sentiment`), providing a numeric measure of how positive or negative the text reads.

3. It builds a `combined_text` field from `name` and `description`, cleans it with regex, and then scans for a curated list of “luxury” keywords (e.g., *luxury*, *executive*, *penthouse*, *private pool*). For each keyword, it creates a binary flag like `has_kw_penthouse`.

4. Finally, it aggregates all these keyword flags into a single `luxury_keyword_hits` feature that counts how many luxury terms appear, and drops the temporary `combined_text` column.

Overall, this function turns unstructured text into length, sentiment, and luxury-signal features that help the model understand how listings are presented.


In [18]:
def add_amenity_features(df):
    df = df.copy()
    
    # --- Full Reference Lists ---
    unique_amenities = [
        'airport transfer', 'private pool', 'pool table', 'piano', 'private hot tub', 'waterfront',
        'mountain view', 'bay view', 'ocean view', 'sea view', 'marina view', 'valley view',
        'vineyard view', 'pool view', 'lake view', 'beach view', 'river view', 'harbor view'
    ]
    luxury_amenities = [
        'dishwasher', 'private entrance', 'private patio or balcony', 'elevator', 'outdoor furniture',
        'bathtub', 'high chair', 'crib', 'bbq grill', 'pets allowed', 'gym',
        'private backyard – fully fenced', 'city skyline view', 'indoor fireplace', 'pool',
        'netflix', 'hot tub', 'heated', 'yoga mat',
        'free parking garage on premises – 1 space',
        'free residential garage on premises – 1 space', 'coffee maker'
    ]
    standard_amenities = [
        'essentials', 'smoke alarm', 'kitchen', 'hangers', 'hair dryer', 'iron',
        'wifi', 'hot water', 'heating', 'microwave', 'dishes and silverware', 'shampoo',
        'refrigerator', 'cooking basics', 'bed linens', 'tv', 'air conditioning',
        'free parking on premises', 'washer', 'stove', 'resort access', 'ev charger'
    ]

    # --- Counts & Ratios ---
    df['count_unique_amenities'] = df['amenities'].apply(lambda x: count_amenities_list(x, unique_amenities))
    df['count_luxury_amenities'] = df['amenities'].apply(lambda x: count_amenities_list(x, luxury_amenities))
    df['count_standard_amenities'] = df['amenities'].apply(lambda x: count_amenities_list(x, standard_amenities))

    # Total Amenities
    if 'amenities_list' in df.columns:
        df['total_amenities'] = df['amenities_list'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    else:
        df['total_amenities'] = df['amenities'].astype(str).str.count(',') + 1
        
    df['luxury_ratio'] = df['count_luxury_amenities'] / (df['total_amenities'] + 1e-6)
    df['standard_ratio'] = df['count_standard_amenities'] / (df['total_amenities'] + 1e-6)

    # --- Premium Flags (Full Mapping) ---
    premium_flags = {
        'has_sauna': 'sauna',
        'has_dishwasher': 'dishwasher',
        'has_private_patio': 'private patio or balcony',
        'has_bathtub': 'bathtub',
        'has_dryer': 'dryer',
        'has_pool': 'pool',
        'has_gym': 'gym',
        'has_fireplace': 'indoor fireplace',
        'has_city_view': 'city skyline view',
        'has_mountain_view': 'mountain view',
        'has_waterfront': 'waterfront',
        'has_refrigerator':'refrigerator'
    }
    for col, kw in premium_flags.items():
        df[col] = df['amenities'].astype(str).str.lower().str.contains(kw.lower(), regex=False).astype(int)

    return df

**`add_amenity_features(df)`**

This function builds detailed amenity-based features using curated lists of unique, luxury, and standard amenities.

1. It defines three reference lists: `unique_amenities` (e.g., waterfront, mountain view), `luxury_amenities` (e.g., private entrance, gym, hot tub), and `standard_amenities` (e.g., wifi, kitchen, heating).

2. Using `count_amenities_list()`, it creates `count_unique_amenities`, `count_luxury_amenities`, and `count_standard_amenities`, which measure how many amenities from each group a listing has.

3. It then calculates `total_amenities` either from `amenities_list` (preferred) or by counting commas in the raw `amenities` string, and derives `luxury_ratio` and `standard_ratio` by dividing the relevant counts by the total amenities.


4. It adds a set of `premium_flags` (e.g., `has_sauna`, `has_dishwasher`, `has_private_patio`, `has_pool`, `has_city_view`, `has_waterfront`, `has_refrigerator`) by checking if those amenity terms appear in the amenities text.

Overall, this function converts raw amenity strings into rich numerical and binary indicators that capture both quantity and quality of amenities.


In [19]:
def add_geolocation_simple(df):
    df = df.copy()
    
    if 'latitude' in df.columns and 'longitude' in df.columns:
        df['distance_to_cbd_km'] = haversine_distance(df['latitude'], df['longitude'])
        df['inv_distance_to_cbd'] = 1 / (1 + df['distance_to_cbd_km'])
        df['is_cbd'] = (df['distance_to_cbd_km'] <= 2).astype(int)
        
        # Zones
        bins = [0, 3, 7, 15, np.inf]
        labels = ['central', 'inner', 'middle', 'outer']
        df['distance_zone'] = pd.cut(df['distance_to_cbd_km'], bins=bins, labels=labels, right=False)
        
        # One-Hot Encode Zones
        zone_dummies = pd.get_dummies(df['distance_zone'], prefix='dist_zone',dtype=int)
        df = pd.concat([df, zone_dummies], axis=1)
        
        # Interactions
        df['dist_entire_home'] = df['distance_to_cbd_km'] * df['is_entire_home']
        if 'dist_zone_central' in df.columns:
            df['entire_home_central'] = ((df['is_entire_home'] == 1) & (df['dist_zone_central'] == 1)).astype(int)
            
    return df

**`add_geolocation_simple(df)`**

This function creates distance-based features from the listing’s coordinates relative to Melbourne CBD.

1. If `latitude` and `longitude` are available, it computes `distance_to_cbd_km` using the Haversine function and then derives `inv_distance_to_cbd` as an inverse-distance feature (closer listings get higher scores).

2. It flags `is_cbd` for listings within 2 km of the CBD.

3. It assigns each listing to a `distance_zone` (central, inner, middle, outer) based on distance bands and then one-hot encodes these zones into dummy variables like `dist_zone_central`.

4. It also creates interaction features such as `dist_entire_home` (distance × entire-home flag) and `entire_home_central` (entire home located in the central zone).

Overall, this function summarises location into interpretable distance zones and interactions that help the model capture how proximity to the CBD affects price.



In [20]:
def add_osm_transit_features(df):
    """
    Uses OSMnx to fetch Melbourne transit stops and calculate distance/count.
    WARNING: Requires Internet and runs somewhat slowly.
    """
    df = df.copy()

    # 1. Fetch Stops
    tags = {"railway": ["station", "tram_stop", "subway_entrance"], "public_transport": "platform"}
    transit_stops = ox.features_from_place("Melbourne, Australia", tags=tags)
    transit_stops = transit_stops[transit_stops.geometry.type == "Point"]
    gdf_stops = transit_stops[['geometry']].copy().to_crs("EPSG:4326")
    
    # 2. Convert Listings to GDF
    gdf_listings = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
    )
    
    # 3. Distance (BallTree)
    listing_coords = np.radians(df[['latitude', 'longitude']].values)
    stop_coords = np.radians(np.array([[p.y, p.x] for p in gdf_stops.geometry]))
    
    tree = BallTree(stop_coords, metric='haversine')
    distances, _ = tree.query(listing_coords, k=1)
    df['min_transit_dist_km'] = distances.flatten() * 6371
    
    # 4. Count within 500m
    gdf_listings['geometry'] = gdf_listings.buffer(500 / 111320)
    
    join = gpd.sjoin(gdf_stops, gdf_listings[['ID', 'geometry']], how='inner', predicate='within')
    counts = join.groupby('ID').size().reset_index(name='num_transit_500m')
    
    df = df.merge(counts, on='ID', how='left')
    df['num_transit_500m'] = df['num_transit_500m'].fillna(0).astype(int)
    
    df['transit_score'] = df['num_transit_500m'] / (1 + df['min_transit_dist_km'])
    
    return df

**`add_osm_transit_features(df)`**

This function uses OpenStreetMap (OSM) data to measure public transport accessibility around each listing.

1. It fetches transit-related points (stations, tram stops, platforms, etc.) in Melbourne using OSMnx and keeps only point geometries.
   
2. It converts listings to a GeoDataFrame and uses a BallTree (with haversine distance) to compute `min_transit_dist_km`, the distance from each listing to the closest transit stop.

3. It buffers each listing by roughly 500 metres, performs a spatial join with transit stops, and counts how many transit stops fall within that buffer, storing the result in `num_transit_500m`.

4. It then creates a `transit_score` that combines both density and distance: `num_transit_500m / (1 + min_transit_dist_km)`.

Overall, this function adds external geospatial context by quantifying how well each listing is served by public transport, which can be a strong driver of price and attractiveness.


In [21]:
def stage_4_engineering(train_df, test_df):
    """
    Master function to apply all feature engineering steps.
    """
    print("--- Stage 4: Feature Engineering Started ---")
    
    # --- A. INDEPENDENT FEATURES (Apply to both separately) ---
    def apply_independent(df):
        df = add_basic_structural_features(df)
        df = add_text_features(df)
        df = add_amenity_features(df)
        df = add_geolocation_simple(df)
        
        # OPTIONAL: Run OSMnx (Heavy computation)
        # Only run if lat/lon exists. Handles errors internally.
        if 'latitude' in df.columns:
            df = add_osm_transit_features(df)
            
        return df

    train_df = apply_independent(train_df)
    
    test_df = apply_independent(test_df)
    
    # --- B. DEPENDENT FEATURES (Train -> Test Leakage Prevention) ---
    
    # Neighborhood Density (Calculate on Train, Map to Test)
    neigh_density = train_df['neighbourhood_cleansed'].value_counts(normalize=True)
    
    train_df['neigh_density'] = train_df['neighbourhood_cleansed'].map(neigh_density)
    test_df['neigh_density'] = test_df['neighbourhood_cleansed'].map(neigh_density).fillna(0)
    
    print("--- Stage 4: Feature Engineering Complete ---")
    return train_df, test_df

**`stage_4_engineering(train_df, test_df)`**

This function is the master controller for all feature engineering and constructs the final model-ready feature set for both the training and test datasets.

1. It first defines an internal helper, `apply_independent(df)`, which applies several independent feature blocks to each dataset separately:
- `add_basic_structural_features()` (reviews, host experience, structure),
- `add_text_features()` (length, sentiment, luxury keywords),
- `add_amenity_features()` (counts, ratios, premium amenity flags), and
- `add_geolocation_simple()` (distance to CBD, zones, interactions).
-  If latitude and longitude are available, it also optionally runs `add_osm_transit_features()` to include public transport accessibility features.

2. This independent pipeline is applied to `train_df` and `test_df` separately so that each dataset gets the same feature logic without cross-contamination.

3. After that, it creates a dependent feature, `neigh_density`, by computing the relative frequency of each `neighbourhood_cleansed` in the **training data only**, and then mapping these density values onto both train and test. Any neighbourhoods not seen in training receive a default density of 0 in the test set.

---
Overall, this function brings together all Stage 4 feature engineering steps, ensuring that both train and test are enriched with consistent, information-rich features while still preventing data leakage by learning neighbourhood-level statistics only from the training data.


## Data Transforming

In [22]:
def get_outlier_caps(df, lower_quantile=0.01, upper_quantile=0.99):
    caps = {}
    
    # Logic is safe: It internally selects only numeric columns
    numeric_cols = df.select_dtypes(include=['int64', 'float64', 'int32']).columns
    cols_to_process = [c for c in numeric_cols if c != 'ID'] 
    
    for col in cols_to_process:
        lower_bound = df[col].quantile(lower_quantile)
        upper_bound = df[col].quantile(upper_quantile)
        caps[col] = (lower_bound, upper_bound)
        
    return caps

def apply_capping(df, caps):
    df = df.copy()
    for col, (lower, upper) in caps.items():
        if col in df.columns:
            df[col] = df[col].clip(lower=lower, upper=upper)
    return df

**get_outlier_caps(df, lower_quantile=0.01, upper_quantile=0.99)**

This function calculates outlier capping thresholds for each numeric column in the dataset.

* It automatically selects only numeric columns (excluding the `ID` column).
* For each numeric variable, it computes the lower and upper quantile values—by default the 1st and 99th percentiles.
* These quantile-based bounds are stored in a dictionary as `(lower_bound, upper_bound)` pairs for every column.

Overall, this function learns the capping thresholds needed to reduce the influence of extreme outliers while preserving the majority of the data’s distribution.

---
 **apply_capping(df, caps)**

This function applies quantile-based outlier clipping to a dataset using pre-computed caps.

* It iterates through each column listed in the `caps` dictionary.
* If the column exists in the dataset, its values are clipped so that anything below the lower bound is set to that bound, and anything above the upper bound is capped at the upper bound.
* A copy of the dataset is returned to avoid modifying the original.

Overall, this function enforces the learned outlier caps on any dataset (train or test), ensuring consistent outlier handling across both.


In [23]:
def stage_5_transformation(train_df, test_df):
    """
    Main Orchestrator for Transformation (Outlier Capping).
    Learns caps from TRAIN numeric data, applies to both.
    """
    print("--- Stage 5: Transformation (Outlier Capping) Started ---")
    
    # 1. Learn Caps from Training Data
    # (The helper function will automatically pick out the numeric columns)
    outlier_caps = get_outlier_caps(train_df, lower_quantile=0.01, upper_quantile=0.99)
    
    # 2. Apply Caps to Training Data
    # (We pass the FULL dataframe. Only the specific numeric columns will be modified)
    train_transformed = apply_capping(train_df, outlier_caps)
    
    # 3. Apply Caps to Test Data
    test_transformed = apply_capping(test_df, outlier_caps)
    
    print("--- Stage 5: Transformation Complete ---")
    return train_transformed, test_transformed

**stage_5_transformation(train_df, test_df)**

This function performs the final transformation step by applying quantile-based outlier capping in a way that prevents data leakage and keeps both datasets consistent.

1. It begins by using `get_outlier_caps()` to learn the lower and upper quantile thresholds for every numeric column **from the training dataset only**, ensuring that test data does not influence the learned caps.

2. It then applies these learned caps to the training data using `apply_capping()`, clipping extreme values to within the 1st and 99th percentile range for each column.

3. The same caps are applied to the test data, guaranteeing that both datasets are treated identically, even if their distributions differ.
---

Overall, this function enforces consistent outlier handling across train and test while preventing leakage by learning all capping thresholds exclusively from the training dataset.





## Data Exporting

In [24]:
def step_6_finalise_and_export(df, filename):
    """
    Final cleanup before saving:
    - Drops intermediate list/text columns that can't be used in modeling.
    - Saves to CSV.
    """
    # Columns to drop because they are raw text, lists, or intermediate helpers
    cols_to_drop = [
        # Raw Text (Use only the extracted numeric features for modeling)
        'description', 'neighborhood_overview', 'host_about', 'host_verifications', 
        'amenities', 'name', 'source','host_name','host_since','host_location','host_response_time',
        'host_neighbourhood','neighbourhood','neighbourhood_cleansed','property_type',
        'room_type','first_review','last_review', 'suburb','municipality',
        
        # Lists (not compatible with standard CSV/Models)
        'amenities_list', 'verifications_list', 
        
        # Intermediate/Helper Columns
        'bath_type', 'bath_type_cleaned', 'host_neighbourhood_cleaned', 'host_location_cleaned',
        'host_neighbourhood_encoded_label', 'host_location_encoded_label',
        'distance_zone','price_tier'
    ]
    
    # Drop only what exists in the dataframe
    existing_drops = [c for c in cols_to_drop if c in df.columns]
    df_final = df.drop(columns=existing_drops)
    
    # Save
    df_final.to_csv(filename, index=False)
    print(f"   > Saved {filename} with shape {df_final.shape}")

**`step_6_finalise_and_export(df, filename)`**

This function performs the final cleanup before exporting the engineered dataset. 
1. It removes all columns that are unsuitable for modelling—such as raw text fields, list-like structures, and intermediate helper variables that were only needed during earlier pipeline stages.

2. After dropping only the columns that actually exist in the dataframe, it saves the cleaned dataset to a CSV file and prints the final shape, ensuring that only the refined, model-ready features remain in the exported output.



## Run Pipeline

In [25]:
# ==========================================
# PIPELINE EXECUTION
# ==========================================

def preprocess_pipeline(train_df: pd.DataFrame, test_df: pd.DataFrame,
                        lookup_csv_path: str = "suburb_municipality_lookup.csv"):
    """
    Orchestrates the full preprocessing pipeline using current stages:
      - Stage 1: Cleaning
      - Stage 2: Imputation
      - Stage 3: Feature Engineering
      - Stage 4: Encoding
      - Stage 5: Transformation (Outlier Capping)
    Returns:
      processed_train_df, processed_test_df
    """

    # --- STAGE 1: CLEANING ---
    print("=== Stage 1: Cleaning (Formatting & Types) ===")
    train_df = step_1_cleaning(train_df, lookup_csv_path=lookup_csv_path)
    test_df  = step_1_cleaning(test_df,  lookup_csv_path=lookup_csv_path)

    # --- STAGE 2: IMPUTATION ---
    # stage_2_imputation currently returns (train_filled, test_filled)
    train_df, test_df = stage_2_imputation(train_df, test_df)

    # --- STAGE 3: ENCODING ---
    # (Semantically Stage 3, function name = stage_3_encoding)
    train_df, test_df = stage_3_encoding(train_df, test_df)

    # --- STAGE 4: FEATURE ENGINEERING ---
    # (Semantically Stage 4, function name = stage_4_engineering)
    train_df, test_df = stage_4_engineering(train_df, test_df)

    # --- STAGE 5: TRANSFORMATION (OUTLIER CAPPING) ---
    train_df, test_df = stage_5_transformation(train_df, test_df)

    print("=== Pipeline: All Stages Complete ===")
    return train_df, test_df


**`preprocess_pipeline(train_df, test_df, lookup_csv_path)`**

This function orchestrates the entire preprocessing workflow by running all stages in sequence on both the training and test datasets.

* **Stage 1 – Cleaning:**
  Calls `step_1_cleaning()` to standardise formats, parse dates, clean text, process amenities and verifications, and merge geolocation lookup information for both train and test.

* **Stage 2 – Imputation:**
  Uses `stage_2_imputation()` to first apply rule-based, row-level fixes, then learn imputation statistics (modes, means/medians, hierarchical mappings) from the **training data only**, and finally apply consistent imputations to both datasets.

* **Stage 3 – Encoding:**
  Runs `stage_3_encoding()` to convert binary flags, ordinal response time, amenities, host location/neighbourhood, bathroom type, property type, and neighbourhood fields into aligned numeric and one-hot encoded features across train and test.

* **Stage 4 – Feature Engineering:**
  Applies `stage_4_engineering()` to add higher-level features, including structural features (reviews, host experience, beds per guest), text-based features (length, sentiment, luxury keywords), amenity richness, distance to CBD, distance zones, transit accessibility, and neighbourhood density.

* **Stage 5 – Transformation (Outlier Capping):**
  Uses `stage_5_transformation()` to learn quantile-based outlier caps from the training data and apply the same caps to both train and test, ensuring consistent outlier handling without leakage.

At the end, the function returns the fully processed `train_df` and `test_df`, ready for modelling.



In [26]:
def run_pipeline():
    """
    Loads datasets, runs the full pipeline, and exports cleaned data
    """

    # 1. Load Raw Data
    train_df = pd.read_csv('train.csv')
    test_df  = pd.read_csv('test.csv')
    print(" Datasets loaded: train.csv, test.csv")
    print(f"   Train shape: {train_df.shape}")
    print(f"   Test  shape: {test_df.shape}")

    # 2. Run Full Preprocessing Pipeline
    train_cleaned, test_cleaned = preprocess_pipeline(train_df, test_df)

    # 3. Export Final Clean Files
    step_6_finalise_and_export(train_cleaned, 'train_cleaned.csv')
    step_6_finalise_and_export(test_cleaned,  'test_cleaned.csv')

    print(" Pipeline complete. Files 'train_cleaned.csv' and 'test_cleaned.csv' are ready for modelling.")
run_pipeline()

 Datasets loaded: train.csv, test.csv
   Train shape: (7000, 61)
   Test  shape: (3000, 60)
=== Stage 1: Cleaning (Formatting & Types) ===
--- Stage 2: Processing missing values ---
--- Stage 2: Imputation Complete ---
--- Stage 3: Encoding Started ---
--- Stage 3: Encoding Complete ---
--- Stage 4: Feature Engineering Started ---
--- Stage 4: Feature Engineering Complete ---
--- Stage 5: Transformation (Outlier Capping) Started ---
--- Stage 5: Transformation Complete ---
=== Pipeline: All Stages Complete ===
   > Saved train_cleaned.csv with shape (7000, 177)
   > Saved test_cleaned.csv with shape (3000, 176)
 Pipeline complete. Files 'train_cleaned.csv' and 'test_cleaned.csv' are ready for modelling.


**`run_pipeline()`**

This function serves as the end-to-end executor for entire data-preprocessing workflow.

* **Step 1 – Load raw data:**
  Reads the original `train.csv` and `test.csv` files into dataframes and prints their shapes so it can quickly verify that the correct data has been loaded.

* **Step 2 – Run the full preprocessing pipeline:**
  Passes both dataframes to `preprocess_pipeline()`, which handles all stages of cleaning, imputation, encoding, feature engineering, and transformation, returning clean, model-ready datasets.

* **Step 3 – Export processed datasets:**
  Uses `step_6_finalise_and_export()` to remove intermediate text/list columns and save the final feature sets as `train_cleaned.csv` and `test_cleaned.csv`.

